# Agent State Machine (ASM) - Quick Start Guide

An **agent** is a wrapper around a finite state machine designed to accomplish a specific task and will be referred to as ASM.

A **StateMachineBuilder** is used to build the ASM from a manifest and a state diagram.

```mermaid
flowchart TD
    A["state_diagram"] -->|Input| B["StateMachineBuilder"]
    A2["state_manifest"] -->|Input| C["StateModel"]
    C -->|Build| B
    B --> D["fsm"]
```


---

## Example: Standard Assistant

In this example, we will demonstrate how to create a simple **Assistant** agent using a state diagram and state manifest.

### a) Define State Diagram

The following is a simple example of an **Assistant** agent using 3 states:

* **INIT:** Collect initial input.
* **GENERATE:** The state where the LLM generates a response.
* **FINAL:** Return final output.

```mermaid
stateDiagram-v2
direction LR
INIT --> GENERATE: next / action
GENERATE --> FINAL: next / action
```


In [1]:
STATE_DIAGRAM = """
    INIT --> GENERATE
    GENERATE --> FINAL
    """

### b) Define State Manifest

The state manifest is a dictionary. 
Each state in the manifest corresponds to a state in the state diagram.


In [2]:
STATE_MANIFEST_V1 = {
        "INIT": {
            "input_data": {
                "name": "Sara",
                "user_message": "Write a one sentence story"
            }
        },
        "GENERATE": {
            "module_path": "gai.asm.states",
            "class_name": "PureActionState",
            "title": "GENERATE",
            "action": "generate",
            "input_data": {
                "llm_config": {
                    "type": "getter",
                    "dependency": "get_llm_config"
                },
            },
            "output_data": [
                "streamer",
                "get_assistant_message"
            ]
        },
        "FINAL": {
            "output_data": ["monologue"]
        }
    }


### c) Create state action

In [3]:
from gai.asm import AsyncStateMachine
from gai.llm.openai import AsyncOpenAI

async def generate_action(state):
    
    llm_config = state.machine.state_bag["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Import data from state_bag
    user_message = state.machine.state_bag.get("user_message", "If you are seeing this, that means I have forgotten to add a user message. Remind me.")
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=[{
            "role":"user",
            "content":user_message
            }],
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk

    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()

### c) Build State Machine

In [4]:
from gai.asm import AsyncStateMachine

with AsyncStateMachine.StateMachineBuilder(STATE_DIAGRAM) as builder:
    fsm = builder.build(
        STATE_MANIFEST_V1,
        get_llm_config=lambda state: {
            "client_type": "gai",
            "model": "ttt",
            "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        },
        generate=generate_action
        )


### d) Run State Machine (INIT->GENERATE)

In [5]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

A young girl named Lily discovers a mystical portal in her grandmother's attic and steps through it, finding herself in a world where time moves backwards.




### d) Continue (GENERATE->FINAL)

In [6]:
await fsm.run_async()
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)
print("Assistant Message:")
fsm.state_bag["get_assistant_message"]()

State History:
State: INIT
- input: {'name': 'Sara', 'user_message': 'Write a one sentence story'}
- output: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715ad0046020>, 'step': 0, 'time': datetime.datetime(2025, 6, 12, 18, 35, 47, 9497)}
--------------------
State: GENERATE
- input: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715ad0046020>, 'step': 1, 'time': datetime.datetime(2025, 6, 12, 18, 35, 47, 9786), 'llm_config': {'client_type': 'gai', 'model': 'ttt', 'url': 'http://gai-llm-svr:12031/gen/v1/chat/completions'}}
- output: {'streamer': <async_generator object generate_action.<locals>.streamer at 0x715aaba1f340>, 'get_assistant_message': <function generate_action.<locals>.<lambda> at 0x715aaba4dab0>, 'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715ac034a950>, 'step': 0, 'time': datetime.datetime(2025, 6, 12, 18, 35, 47, 650188)}
--------------------
State: FINAL
- input: {'streamer': <asyn

"A young girl named Lily discovers a mystical portal in her grandmother's attic and steps through it, finding herself in a world where time moves backwards."

---

## Example: Standard Assistant (Part 2)

Same example but with an additional state to demonstrate context management by monologue messages.

```mermaid
stateDiagram-v2
direction LR
INIT --> GENERATE
GENERATE --> CONTINUE
CONTINUE --> FINAL
```


In [7]:
STATE_DIAGRAM = """
    INIT --> GENERATE
    GENERATE --> CONTINUE
    CONTINUE --> FINAL
    """
    
STATE_MANIFEST_V1 = {
        "INIT": {
            "input_data": {
                "name": "Sara",
                "user_message": "Write a one sentence story"
            }
        },
        "GENERATE": {
            "module_path": "gai.asm.states",
            "class_name": "PureActionState",
            "title": "GENERATE",
            "action": "generate_action",
            "input_data": {
                "llm_config": {
                    "type": "getter",
                    "dependency": "get_llm_config"
                },
            },
            "output_data": [
                "streamer",
                "get_assistant_message"
            ]
        },
        "CONTINUE": {
            "module_path": "gai.asm.states",
            "class_name": "PureActionState",
            "title": "CONTINUE",
            "action": "continue_action",
            "input_data": {
                "llm_config": {
                    "type": "getter",
                    "dependency": "get_llm_config"
                },
            },
            "output_data": [
                "streamer",
                "get_assistant_message"
            ]
        },
        "FINAL": {
            "output_data": ["monologue"]
        }
    }

from gai.llm.openai import AsyncOpenAI

async def generate_action(state):
    
    llm_config = state.machine.state_bag["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Import data from state_bag
    user_message = state.machine.state_bag.get("user_message", "If you are seeing this, that means I have forgotten to add a user message. Remind me.")
    monologue = state.machine.state_bag["monologue"]
    monologue.add_user_message(state=state,content=user_message)
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=monologue.list_chat_messages(),
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk
        monologue.add_assistant_message(state=state,content=assistant_message)
    state.machine.state_bag["monologue"] = monologue
    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()

async def continue_action(state):
    
    llm_config = state.machine.state_bag["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Import data from state_bag
    monologue = state.machine.state_bag["monologue"]
    monologue.add_user_message(state=state,content="Please continue.")
    
    # Execute
    
    response = await client.chat.completions.create(
        model=llm_config["model"],
        messages=monologue.list_chat_messages(),
        max_tokens=50,
        stream=True
    )
    
    assistant_message = ""
    async def streamer():
        nonlocal assistant_message
        async for chunk in response:
            chunk = chunk.choices[0].delta.content
            if isinstance(chunk,str) and chunk:
                assistant_message += chunk
                yield chunk
        monologue.add_assistant_message(state=state,content=assistant_message)
    state.machine.state_bag["monologue"] = monologue
    state.machine.state_bag["get_assistant_message"] = lambda: assistant_message
    state.machine.state_bag["streamer"] = streamer()
    
from gai.asm import AsyncStateMachine

with AsyncStateMachine.StateMachineBuilder(STATE_DIAGRAM) as builder:
    fsm = builder.build(
        STATE_MANIFEST_V1,
        get_llm_config=lambda state: {
            "client_type": "gai",
            "name": "dolphin_llama:exl2",
            "model": "ttt",
            "url": "http://gai-llm-svr:12031/gen/v1/chat/completions"
        },
        generate_action=generate_action,
        continue_action=continue_action
        )


### d) Run State Machine (INIT->GENERATE)

In [8]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

A lone traveler sat on the edge of the world, watching a crimson sun sink slowly into the vast ocean, painting the sky with strokes of gold and purple, as a gentle breeze whispered ancient secrets of forgotten lands.




### c) Run State Machine (GENERATE->CONTINUE)

In [9]:
await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    print(chunk,end='',flush=True)
print("\n\n")

The traveler, a young woman with eyes like the sea and hair that danced in the wind, closed her eyes and let the colors and whispers wash over her, carrying her back to memories of childhood tales told by her grandmother, a wise woman who had sailed




### e) END (GENERATE->FINAL)

In [10]:
await fsm.run_async()
print("State History:")
for state in fsm.state_history:
    print(f"State: {state['state']}")
    print(f"- input: {state['input']}")
    print(f"- output: {state['output']}")
    print("-" * 20)
print("Assistant Message:")
fsm.state_bag["get_assistant_message"]()

State History:
State: INIT
- input: {'name': 'Sara', 'user_message': 'Write a one sentence story'}
- output: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715aab10f1f0>, 'step': 0, 'time': datetime.datetime(2025, 6, 12, 18, 36, 8, 16780)}
--------------------
State: GENERATE
- input: {'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715aab10f1f0>, 'step': 1, 'time': datetime.datetime(2025, 6, 12, 18, 36, 8, 17159), 'llm_config': {'client_type': 'gai', 'name': 'dolphin_llama:exl2', 'model': 'ttt', 'url': 'http://gai-llm-svr:12031/gen/v1/chat/completions'}}
- output: {'streamer': <async_generator object generate_action.<locals>.streamer at 0x715aaba89d40>, 'get_assistant_message': <function generate_action.<locals>.<lambda> at 0x715aababedd0>, 'name': 'Assistant', 'monologue': <gai.asm.monologue.Monologue object at 0x715aab10cbb0>, 'step': 0, 'time': datetime.datetime(2025, 6, 12, 18, 36, 8, 39524)}
--------------------
State: CONTI

'The traveler, a young woman with eyes like the sea and hair that danced in the wind, closed her eyes and let the colors and whispers wash over her, carrying her back to memories of childhood tales told by her grandmother, a wise woman who had sailed'